In [1]:
## Funciones de Usuario (udfs)
##   Apache Pig permite la ejecución de funciones escritas en Java, Jython (http://www.jython.org/index.html), JavaScript, Ruby, Groovy y Python. 
##   Información detallada de la ejecución de udfs en estos lenguajes puede ser obtenida en https://pig.apache.org/docs/latest/udf.html#python-udfs.
##   Véase http://www.jython.org/index.html. Para su uso, el script de Python debe estar ubicado en el directorio de trabajo. Para poder usar Jython, hay dos opciones; 
##   en la primera se debe indicar la ubicación del inteprete en la variable PIG_CLASSPATH. En algunos casos es necesario establacer la variable PYTHONPATH para que 
##   Pig pueda compilar las udfs.

In [2]:
%load_ext bigdata

In [3]:
%pig_start

In [4]:
%timeout 300

In [5]:
%%writefile udf.tsv
A	B	1
C	D	2
E	F	3
G	H	4
I	J	5

Writing udf.tsv


In [6]:
!hadoop fs -put udf.tsv

In [7]:
%%writefile pigudf.py

#from pig_util import outputSchema

@outputSchema("as:int")
def square(num):
    if num == None:
        return None
    return ((num) * (num))

@outputSchema("word:chararray")
def concatenar(word1, word2):
    return (word1 + word2)

Writing pigudf.py


In [8]:
%%pig
--
-- registra las funciones en el script `pigudf.py`.
--
REGISTER /Users/jdvelasq/jython2.7.0/jython.jar
REGISTER 'pigudf.py' using jython as myudf;

u = LOAD 'udf.tsv' USING PigStorage()
    AS (c1:CHARARRAY,
        c2:CHARARRAY,
        c3:INT);

b = FOREACH u GENERATE myudf.concatenar(c1, c2), myudf.square(c3);
DUMP b;

 --
 -- registra las funciones en el script `pigudf.py`.
 --
 REGISTER /Users/jdvelasq/jython2.7.0/jython.jar
2019-11-28 01:52:33,225 [main] ERROR org.apache.pig.tools.grunt.Grunt - ERROR 101: file '/Users/jdvelasq/jython2.7.0/jython.jar' does not exist.
Details at logfile: /datalake/pig_1574905866736.log
 REGISTER 'pigudf.py' using jython as myudf;
 u = LOAD 'udf.tsv' USING PigStorage()
    AS (c1:CHARARRAY,
        c2:CHARARRAY,
        c3:INT);
 b = FOREACH u GENERATE myudf.concatenar(c1, c2), myudf.square(c3);
 DUMP b;
2019-11-28 01:52:38,624 [main] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2019-11-28 01:52:38,803 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.reduce.markreset.buffer.percent is deprecated. Instead, use mapreduce.reduce.markreset.buffer.percent
2019-11-28 01:52:38,806 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.output.compress is deprecated. Instead, use mapredu

In [9]:
!rm pigudf.py pig_*  udf.tsv